## clinical variable retrieval codebase.
each querey is referencing a sql querey on my computer for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. it's inefficient but works. 

In [11]:
#6-25-18
#the streamlined version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
###THIS IS STILL A WORK IN PROGRESS

time: 816 µs


In [3]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [8]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/final_pt_df2_5-04-18.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 70.1 ms


### todo 5/8/2018:
    would be nice to ping server in each querey for only patients or icustay_id in the final cohort,
    would greatly reduce querey time.

# todo 5/29/2018:
    
    put all of these python scripts into .py files and make a runall for it. 

## Fetch all responses within one Client session first

In [13]:
#I need help understanding what this is doing, can you elaborate?
#what are the fetch and run functions? This is used in pinging a api, right?

time: 626 µs


In [5]:
import asyncio
from aiohttp import ClientSession
import time
import json

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def run(url_lst):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    start = time.perf_counter()
    
    async with ClientSession() as session:
        for url in url_lst:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
#         print(responses)

    duration = time.perf_counter() - start
    print('total time: {}'.format(duration))
    return responses

## Access MIMIC database and convert it to dataframe in Pandas

In [173]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

time: 2.66 s


In [5]:
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

time: 4.81 ms


In [6]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

time: 5.67 ms


## Results in .csv

In [ ]:
#where are these tables? i don't see any code generating this? 

In [95]:
index = [] 
index.append(echoTable.columns)
index.append(heightFirstDayTable.columns)
index.append(weightFirstDayTable.columns)
index.append(vitalsFirstDayTable.columns)
index.append(labsFirstDayTable.columns)
index.append(bgFirstDayTable.columns)
index.append(gcsFirstDayTable.columns)
index.append(rrtFirstDayTable.columns)
index.append(urineFirstDayTable.columns)
pd.DataFrame(index).to_csv('index.csv')


NameError: name 'vitalsFirstDay' is not defined

time: 46.7 ms


In [99]:
# index = [] 
# index.append(echoDf.columns)
# index.append(heightFirstDayDf.columns)
# index.append(weightFirstDayDf.columns)
# index.append(vitalsFirstDayDf.columns)
# index.append(labsFirstDayDf.columns)
# index.append(bgFirstDayDf.columns)
# index.append(gcsFirstDayDf.columns)
# index.append(rrtFirstDayDf.columns)
# index.append(urineFirstDayDf.columns)
pd.DataFrame(echoDf).to_csv('clinical_csv/echoDf.csv')
pd.DataFrame(heightFirstDayDf).to_csv('clinical_csv/heightFirstDayDf.csv')
pd.DataFrame(weightFirstDayDf).to_csv('clinical_csv/weightFirstDayDf.csv')
pd.DataFrame(vitalsFirstDayDf).to_csv('clinical_csv/vitalsFirstDayDf.csv')
pd.DataFrame(labsFirstDayDf).to_csv('clinical_csv/labsFirstDayDf.csv')
pd.DataFrame(bgFirstDayDf).to_csv('clinical_csv/bgFirstDayDf.csv')
pd.DataFrame(gcsFirstDayDf).to_csv('clinical_csv/gcsFirstDayDf.csv')
pd.DataFrame(rrtFirstDayDf).to_csv('clinical_csv/rrtFirstDayDf.csv')
pd.DataFrame(urineFirstDayDf).to_csv('clinical_csv/urineFirstDayDf.csv')


time: 2.75 s


# extracting clinical data for our patients

## Echodata in Noteevents

In [1]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/echodata.sql', 'r') 
echodata = f.read()
cur.execute('rollback')
cur.execute(echodata)
echoTable = sql_exe_show('select * from echodata;')
echoDf= sql_to_df_patients(echoTable)
echoDf[0:500]

NameError: name 'cur' is not defined

In [52]:
echoTable[0:500]

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59653,31038,174978.0,2102-06-14,2102-06-14 13:30:00,Endocarditis.,74,165,2.00,102/72,102,72,86,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
1,59654,70150,156140.0,2191-05-10,2191-05-10 15:08:00,"CHF, A-FIB",66,263,2.25,107/81,107,81,95,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
2,59655,54190,188571.0,2158-02-16,2158-02-16 13:48:00,Cerebrovascular event/TIA. Streptococcal bacte...,72,187,2.07,147/87,147,87,90,Inpatient,Portable TEE (Complete),Full Doppler and color Doppler,None,Adequate
3,59656,5771,185291.0,2173-09-02,2173-09-02 09:47:00,78 year-old man with atrial fibrillation off a...,66,130,1.67,163/63,163,63,80,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
4,59657,80030,100442.0,2119-06-07,2119-06-07 14:59:00,Left ventricular function.,63,126,1.59,88/53,88,53,115,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
5,59658,65696,167705.0,2196-09-16,2196-09-16 01:19:00,Left ventricular function. Myocardial infarction.,73,200,2.15,100/42,100,42,89,Inpatient,Portable TTE (Focused views),Limited Doppler and color Doppler,None,Adequate
6,59659,82208,188268.0,2200-10-22,2200-10-22 15:14:00,"Acute MR worsening,ESRD, on HD, NSLS lung Ca, ...",59,136,1.57,126/61,126,61,95,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
7,59660,82208,NaN,2200-01-02,2200-01-02 11:23:00,Congestive heart failure. Hypertension. Shortn...,59,143,1.60,170/82,170,82,92,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
8,59669,15472,NaN,2176-05-28,2176-05-28 08:58:00,Left ventricular function. Shortness of breath...,72,200,2.13,122/90,122,90,None,Inpatient,TTE(Focused views),Focused pulse and color flow,None,Suboptimal
9,59670,15472,118185.0,2175-09-19,2175-09-19 09:58:00,Left ventricular function.,None,None,None,123/76,123,76,None,Inpatient,Portable TEE(Complete),Complete pulse and color flow,None,Adequate


time: 68.6 ms


## Weight

In [12]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/weight.sql', 'r') 
weightSQL = f.read()
cur.execute('rollback')
cur.execute(weightSQL)
weightFirstDayTable = sql_exe_show('select * from weightfirstday;')
weightFirstDayDf= sql_to_df_icu(weightFirstDayTable)
weightFirstDayDf[0:500]


NameError: name 'icustay_id' is not defined

time: 1.86 s


## Height

In [9]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/height.sql', 'r')
heightSQL = f.read()
cur.execute('rollback') #not sure what this does
cur.execute(heightSQL)
heightFirstDayTable = sql_exe_show('select * from heightfirstday;')
heightFirstDayDf= sql_to_df_icu(heightFirstDayTable)
heightFirstDayDf[0:500]




NameError: name 'cur' is not defined

time: 11.5 ms


# Labs

In [87]:
# -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit


f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/labs.sql', 'r')
labsSQL = f.read()
cur.execute('rollback')
cur.execute(labsSQL)
labsFirstDayTable = sql_exe_show('select * from labsfirstday;')
labsFirstDayDf= sql_to_df_icu(labsFirstDayTable)
labsFirstDayDf[0:500]



,subject_id,hadm_id,icustay_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,...,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
2,4,185777,294638,15.0,15.0,2.8,2.8,NaN,NaN,21.0,...,1.1,1.1,12.8,12.8,141.0,141.0,10.0,10.0,9.7,9.7
17,19,109235,273430,15.0,15.0,NaN,NaN,NaN,NaN,22.0,...,1.2,1.3,13.6,14.0,138.0,138.0,25.0,25.0,10.1,10.1
20,21,111970,216859,15.0,19.0,2.1,2.1,1.0,1.0,23.0,...,1.7,1.8,18.1,19.1,137.0,140.0,49.0,50.0,30.5,41.2
25,25,129635,203487,7.0,19.0,3.2,3.2,NaN,NaN,21.0,...,1.0,1.1,12.8,13.0,134.0,139.0,34.0,54.0,11.6,13.0
26,26,197661,244882,12.0,14.0,NaN,NaN,NaN,NaN,24.0,...,1.5,2.4,15.2,18.9,139.0,143.0,31.0,33.0,8.2,8.2
30,31,128652,254478,9.0,10.0,2.7,2.7,NaN,NaN,24.0,...,1.1,1.2,13.0,13.2,128.0,133.0,11.0,13.0,6.9,6.9
32,33,176176,296681,9.0,16.0,3.9,3.9,6.0,12.0,25.0,...,1.2,1.2,13.3,13.3,137.0,140.0,17.0,21.0,26.4,33.5
61,61,176332,252348,11.0,11.0,NaN,NaN,1.0,1.0,23.0,...,0.9,0.9,11.6,11.6,140.0,140.0,17.0,17.0,1.6,1.6
63,62,116009,216609,9.0,13.0,2.7,2.7,7.0,21.0,22.0,...,1.1,1.2,13.0,13.7,136.0,139.0,19.0,22.0,2.0,14.4
65,64,172056,232593,8.0,15.0,2.3,2.3,19.0,19.0,16.0,...,1.1,1.6,12.9,15.6,135.0,140.0,12.0,26.0,12.2,18.1


time: 3min 15s


In [24]:
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/durations/ventilation-durations.sql', 'r')
ventsettingsSQL = f.read()
cur.execute('rollback')
cur.execute(ventsettingsSQL)
ventsettingsTable = sql_exe_show('select * from ventsettings;')
ventsettingsDf= sql_to_df_icu(ventsettingsTable)
ventsettingsDf[0:500]

,icustay_id,charttime,mechvent,oxygentherapy,extubated,selfextubated
1,282983.0,2185-01-14 14:30:00,1,0,0,0
6,255356.0,2156-04-26 18:00:00,0,1,0,0
13,249403.0,2158-10-24 01:49:00,1,0,0,0
14,288354.0,2103-11-04 09:40:00,0,1,0,0
16,273995.0,2177-07-11 02:00:00,1,0,0,0
20,226799.0,2153-08-06 11:03:00,1,0,0,0
22,232333.0,2120-08-22 12:00:00,0,0,0,0
26,215903.0,2175-06-10 12:59:00,1,0,0,0
27,258962.0,2134-08-29 00:33:00,1,0,0,0
33,226799.0,2153-08-22 02:43:00,1,0,0,0


time: 4min 4s


## Vital Sign

In [88]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/vitalsign.sql', 'r')
vitalsSQL = f.read()
cur.execute('rollback')
cur.execute(vitalsSQL)
vitalsFirstDayTable = sql_exe_show('select * from vitalsfirstday;')
vitalsFirstDayDf= sql_to_df_icu(vitalsFirstDayTable)
vitalsFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,heartrate_mean,sysbp_min,sysbp_max,sysbp_mean,diasbp_min,...,resprate_mean,tempc_min,tempc_max,tempc_mean,spo2_min,spo2_max,spo2_mean,glucose_min,glucose_max,glucose_mean
2,4,185777,294638,74.0,111.0,89.217391,97.0,139.0,118.000000,55.0,...,25.352941,36.333334,37.444445,36.844445,91.0,100.0,97.500000,153.0,266.0,199.666667
15,19,109235,273430,76.0,114.0,98.500000,118.0,163.0,146.350000,44.0,...,20.500000,36.611112,37.277777,36.851852,96.0,100.0,98.150000,139.0,139.0,139.000000
18,21,111970,216859,66.0,83.0,74.971429,78.0,125.0,97.787879,21.0,...,16.885714,36.944444,37.944446,37.411111,79.0,100.0,96.323529,138.0,213.0,171.000000
23,25,129635,203487,49.0,104.0,72.906250,62.0,125.0,100.000000,28.0,...,16.031250,35.722224,36.777776,36.333333,91.0,100.0,97.096774,61.0,364.0,164.766667
24,26,197661,244882,60.0,65.0,62.476190,89.0,111.0,98.047619,24.0,...,19.714286,37.222222,37.722223,37.541666,96.0,98.0,97.090909,118.0,184.0,136.600000
28,31,128652,254478,48.0,54.0,51.230769,85.0,146.0,114.153846,48.0,...,12.066667,36.833335,37.055554,36.958333,97.0,100.0,98.714286,101.0,119.0,108.142857
30,33,176176,296681,58.0,100.0,66.800000,96.0,129.0,113.920000,41.0,...,17.400000,36.611112,37.444445,37.013889,92.0,99.0,95.440000,92.0,115.0,103.500000
57,61,176332,252348,73.0,116.0,94.000000,81.0,116.0,93.565217,40.0,...,20.814815,36.666667,38.888889,38.111112,97.0,100.0,98.555556,95.0,95.0,95.000000
59,62,116009,216609,45.0,86.0,55.840909,87.0,127.0,100.545455,58.0,...,21.909091,35.388887,36.444444,36.034722,93.0,100.0,98.210526,135.0,221.0,178.055556
61,64,172056,232593,88.0,113.0,100.259259,104.0,118.0,110.826087,41.0,...,17.037037,36.444444,39.111112,38.148148,94.0,100.0,97.666667,111.0,188.0,146.000000


time: 8min 58s


In [ ]:
vitalsFirstDayDf.loc[vitalsFirstDayDf.loc[:,'subject_id']=='31038',:]

## Glasgow Coma Scale

In [ ]:
# -- ITEMIDs used:

# -- CAREVUE
# --    723 as GCSVerbal
# --    454 as GCSMotor
# --    184 as GCSEyes

# -- METAVISION
# --    223900 GCS - Verbal Response
# --    223901 GCS - Motor Response
# --    220739 GCS - Eye Opening

# -- The code combines the ITEMIDs into the carevue itemids, then pivots those
# -- So 223900 is changed to 723, then the ITEMID 723 is pivoted to form GCSVerbal

# -- Note:
# --  The GCS for sedated patients is defaulted to 15 in this code.
# --  This is in line with how the data is meant to be collected.
# --  e.g., from the SAPS II publication:
# --    For sedated patients, the Glasgow Coma Score before sedation was used.
# --    This was ascertained either from interviewing the physician who ordered the sedation,
# --    or by reviewing the patient's medical record.



In [89]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/gcs1.sql', 'r')
gcsSQL = f.read()
cur.execute('rollback')
cur.execute(gcsSQL)
gcsFirstDayTable = sql_exe_show('select * from gcsfirstday;')
gcsFirstDayDf= sql_to_df_icu(gcsFirstDayTable)
gcsFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag
7,28448,177527,200012,15.0,6.0,5.0,4.0,0.0
12,21789,112486,200019,7.0,5.0,1.0,1.0,0.0
25,9960,159243,200036,3.0,1.0,1.0,1.0,0.0
39,25225,147080,200055,14.0,6.0,5.0,3.0,0.0
41,28494,199133,200059,15.0,6.0,0.0,3.0,1.0
45,99052,129142,200063,15.0,1.0,0.0,1.0,1.0
46,19891,112840,200065,15.0,1.0,0.0,1.0,1.0
60,61871,124231,200087,15.0,6.0,5.0,4.0,0.0
64,30284,136554,200095,15.0,4.0,0.0,4.0,1.0
67,52619,175374,200099,14.0,6.0,5.0,3.0,0.0


time: 1min 27s


## Renal replacement therapy

In [90]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/rrt.sql', 'r')
rrtSQL = f.read()
cur.execute('rollback')
cur.execute(rrtSQL)
rrtFirstDayTable = sql_exe_show('select * from rrtfirstday;')
rrtFirstDayDf= sql_to_df_icu(rrtFirstDayTable)
rrtFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,rrt
7,28448,177527,200012,0
12,21789,112486,200019,0
25,9960,159243,200036,0
39,25225,147080,200055,1
41,28494,199133,200059,0
45,99052,129142,200063,0
46,19891,112840,200065,1
60,61871,124231,200087,0
64,30284,136554,200095,0
67,52619,175374,200099,0


time: 20.4 s


In [ ]:
#attempting to write a fxn to perform all this, but the sql_exe_show line requires more customization
# Def file_to_df(filename_no_extension):
#     f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/%s.sql', 'r') %(filename_no_extension)
#     filenameSQL = f.read()
#     cur.execute('rollback')
#     cur.execute(filenameSQL)
#     filenameFirstDayTable = sql_exe_show('select * from vitalsfirstday;')
#     vitalsFirstDayDf= sql_to_df_icu(vitalsFirstDayTable)
#     vitalsFirstDayDf[0:500]

## Urine Output

In [ ]:
#Purpose: Create a view of the urine output for each ICUSTAY_ID over the first 24 hours.

In [15]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/urine.sql', 'r')
urineSQL = f.read()
cur.execute('rollback')
cur.execute(urineSQL)
urineFirstDayTable = sql_exe_show('select * from uofirstday;')
# urineFirstDayDf = sql_to_df_icu(urineFirstDayTable)
# urineFirstDayDf[0:500]



time: 3.98 s


## Blood Gas Test

In [ ]:
# -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS

# -- things to check:
# --  when a mixed venous/arterial blood sample are taken at the same time, is the store time different?


In [92]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/bloodgas.sql', 'r')
bgSQL = f.read()
cur.execute('rollback')
cur.execute(bgSQL)
bgFirstDayTable = sql_exe_show('select * from bloodgasfirstday;')
bgFirstDayDf = sql_to_df_icu(bgFirstDayTable)
bgFirstDayDf[0:500]


NameError: name 'bgFirstDay' is not defined

time: 1min 56s


,subject_id,hadm_id,icustay_id,charttime,specimen,aado2,baseexcess,bicarbonate,totalco2,carboxyhemoglobin,...,peep,ph,po2,potassium,requiredo2,sodium,temperature,tidalvolume,ventilationrate,ventilator
20,4,185777,294638,2191-03-15 21:23:00,ART,NaN,NaN,NaN,21.0,NaN,...,NaN,7.47,74.0,NaN,NaN,NaN,38.9,NaN,None,None
62,19,109235,273430,NaT,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
82,21,111970,216859,2135-01-30 19:18:00,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
101,25,129635,203487,2160-11-02 06:05:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.39,102.0,NaN,NaN,NaN,NaN,NaN,None,None
102,25,129635,203487,2160-11-02 07:59:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.33,46.0,NaN,NaN,NaN,NaN,NaN,None,None
103,25,129635,203487,2160-11-02 08:51:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.34,96.0,NaN,NaN,NaN,37.0,NaN,None,None
104,25,129635,203487,2160-11-02 09:23:00,MIX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
105,25,129635,203487,2160-11-02 09:25:00,VEN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
106,25,129635,203487,2160-11-02 10:36:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.28,167.0,NaN,NaN,NaN,NaN,NaN,None,None
107,25,129635,203487,2160-11-02 11:06:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.29,71.0,NaN,NaN,NaN,NaN,NaN,None,None


time: 58.6 ms


bgFirstDayDf
urineFirstDayDf
rrtFirstDayDf
gcsFirstDayDf
vitalsFirstDayDf
labsFirstDayDf
heightFirstDayDf


# clinical variable QC

In [103]:
lab_bg_overlap_list=list(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(list(
    bgFirstDayDf['subject_id'].unique()))]['subject_id'].unique())

time: 22.1 ms


In [131]:
#bgFirstDayDf
#(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(lab_bg_overlap_list)])
(labsFirstDayDf[labsFirstDayDf['subject_id']==99912])[['potassium_min','potassium_max','icustay_id','subject_id','hadm_id']]

,potassium_min,potassium_max,icustay_id,subject_id,hadm_id
61504,3.9,4.6,236183,99912,189380


time: 10.1 ms


In [130]:
#5-9-18
#(bgFirstDayDf[bgFirstDayDf['subject_id']==(lab_bg_overlap_list)])
#(bgFirstDayDf[bgFirstDayDf['subject_id']==4])[['bicarbonate','specimen','charttime']]
(bgFirstDayDf[bgFirstDayDf['subject_id']==99912])[['potassium','specimen','charttime','icustay_id','subject_id','hadm_id']]
#bgFirstDayDf
#99966 and 99912 subject id has potassium


#qc, checking overlapping categories between bgFirstDayDf and labsFirstDatDf
#subject_id has potassium 4.7 on 2191-08-23 21:38:00 in bgFirstDayDf 
#subject_id has potassium 4.4-4.9 in labsFirstDayDf. 


(ie.intime - interval '6' hour) AND (ie.intime + interval '1' day)


#labs: # -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit.
#these are from labevents
#labs taken between 6 hours prior to admin and 1 day after admission. 

#bloodgas: # -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS
#labs taken between 6 hours prior to admin and 1 day after admission. 

#ok so labs uses itemid of either 50822 & 50971 where as bloodgas only uses 50822. 
#50822 is postassium, whole blood (category bloodgas) while 50971 is potassium in blood (category chemistry)

,potassium,specimen,charttime,icustay_id,subject_id,hadm_id
243343,4.4,None,2153-04-13 13:37:00,236183,99912,189380
243344,NaN,None,2153-04-13 16:26:00,236183,99912,189380
243345,NaN,None,2153-04-13 16:50:00,236183,99912,189380
243346,NaN,None,2153-04-13 18:36:00,236183,99912,189380
243347,NaN,None,2153-04-13 20:53:00,236183,99912,189380
243348,NaN,None,2153-04-13 21:54:00,236183,99912,189380
243349,NaN,None,2153-04-14 00:20:00,236183,99912,189380
243350,NaN,None,2153-04-14 01:50:00,236183,99912,189380
243351,NaN,None,2153-04-14 04:32:00,236183,99912,189380
243352,NaN,None,2153-04-14 06:38:00,236183,99912,189380


time: 19.2 ms


# altering the sql files to search for all instances within 48 hours

In [7]:
#labs
labs_all48_sql = query_schema + """

SELECT *

FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, le.charttime
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
	  --WHEN ie.subject_id not in (24915,24942,24968,24776,24714,24824) THEN null
    ELSE le.valuenum
    END AS valuenum

  FROM mimiciii.icustays ie
	--where ie.subject_id in (24915,24942,24968,24776,24714,24824)
  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (ie.intime - interval '6' hour) AND (ie.intime + interval '2' day)
	AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt;

    """.format(tuple(patients))
    
labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    


#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)
#will need to do this for all of my sql queres.


###guessing this isn't correct yet. need to adjust by removing the group_by's. addin charttime, and remove the max/min categories.

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



NameError: name 'patients' is not defined

time: 144 ms


In [195]:
labs_all48_df

,subject_id,hadm_id,icustay_id,charttime,label,valuenum
0,4,185777,294638,2191-03-15 21:23:00,HEMATOCRIT,32.0
1,4,185777,294638,2191-03-15 21:23:00,HEMOGLOBIN,10.5
2,4,185777,294638,2191-03-15 21:23:00,LACTATE,2.1
3,4,185777,294638,2191-03-16 05:42:00,ALBUMIN,2.8
4,4,185777,294638,2191-03-16 05:42:00,ANION GAP,15.0
5,4,185777,294638,2191-03-16 05:42:00,BICARBONATE,21.0
6,4,185777,294638,2191-03-16 05:42:00,BILIRUBIN,1.9
7,4,185777,294638,2191-03-16 05:42:00,CHLORIDE,108.0
8,4,185777,294638,2191-03-16 05:42:00,CREATININE,0.5
9,4,185777,294638,2191-03-16 05:42:00,GLUCOSE,183.0


time: 32.3 ms


In [179]:
#blood gas, updated

bg_all48_sql = query_schema + """
select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME, label, valuenum

from
( -- begin query that extracts the data
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
      , case
        when itemid = 50800 then 'SPECIMEN'
        when itemid = 50801 then 'AADO2'
        when itemid = 50802 then 'BASEEXCESS'
        when itemid = 50803 then 'BICARBONATE'
        when itemid = 50804 then 'TOTALCO2'
        when itemid = 50805 then 'CARBOXYHEMOGLOBIN'
        when itemid = 50806 then 'CHLORIDE'
        when itemid = 50808 then 'CALCIUM'
        when itemid = 50809 then 'GLUCOSE'
        when itemid = 50810 then 'HEMATOCRIT'
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 50812 then 'INTUBATED'
        when itemid = 50813 then 'LACTATE'
        when itemid = 50814 then 'METHEMOGLOBIN'
        when itemid = 50815 then 'O2FLOW'
        when itemid = 50816 then 'FIO2'
        when itemid = 50817 then 'SO2' -- OXYGENSATURATION
        when itemid = 50818 then 'PCO2'
        when itemid = 50819 then 'PEEP'
        when itemid = 50820 then 'PH'
        when itemid = 50821 then 'PO2'
        when itemid = 50822 then 'POTASSIUM'
        when itemid = 50823 then 'REQUIREDO2'
        when itemid = 50824 then 'SODIUM'
        when itemid = 50825 then 'TEMPERATURE'
        when itemid = 50826 then 'TIDALVOLUME'
        when itemid = 50827 then 'VENTILATIONRATE'
        when itemid = 50828 then 'VENTILATOR'
        else null
        end as label
        , charttime
        , value
        -- add in some sanity checks on the values
        , case
          when valuenum <= 0 then null
          when itemid = 50810 and valuenum > 100 then null -- hematocrit
          when itemid = 50816 and valuenum > 100 then null -- FiO2
          when itemid = 50817 and valuenum > 100 then null -- O2 sat
          when itemid = 50815 and valuenum >  70 then null -- O2 flow
          when itemid = 50821 and valuenum > 800 then null -- PO2
           -- conservative upper limit
        else valuenum
        end as valuenum

    from mimiciii.icustays ie
    left join mimiciii.labevents le
      on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
      and le.charttime between (ie.intime - interval '6' hour) and (ie.intime + interval '2' day)
  	  and ie.subject_id in {} --!!THIS IS THE code to get for only specific pt. 
      and le.ITEMID in
      -- blood gases
      (
        50800, 50801, 50802, 50803, 50804, 50805, 50806, 50807, 50808, 50809
        , 50810, 50811, 50812, 50813, 50814, 50815, 50816, 50817, 50818, 50819
        , 50820, 50821, 50822, 50823, 50824, 50825, 50826, 50827, 50828
        , 51545
      )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME, label, valuenum
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME, label, valuenum
; """.format(tuple(patients))
    
bg_all48_df=pd.read_sql_query(bg_all48_sql,conn)    
bg_all48_df= bg_all48_df[bg_all48_df['label'].notnull()]# 44451 null rows removed. 
#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)

time: 3min 34s


In [163]:
# #blood gas,  mid-modification, saved for reference

# bg_all48_sql = query_schema + """
# -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS

# -- things to check:
# --  when a mixed venous/arterial blood sample are taken at the same time, is the store time different?

# --DROP MATERIALIZED VIEW IF EXISTS bloodgasfirstday CASCADE;
# --create materialized view bloodgasfirstday as
# select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME

# , max(case when label = 'SPECIMEN' then value else null end) as SPECIMEN
# , max(case when label = 'AADO2' then valuenum else null end) as AADO2
# , max(case when label = 'BASEEXCESS' then valuenum else null end) as BASEEXCESS
# , max(case when label = 'BICARBONATE' then valuenum else null end) as BICARBONATE
# , max(case when label = 'TOTALCO2' then valuenum else null end) as TOTALCO2
# , max(case when label = 'CARBOXYHEMOGLOBIN' then valuenum else null end) as CARBOXYHEMOGLOBIN
# , max(case when label = 'CHLORIDE' then valuenum else null end) as CHLORIDE
# , max(case when label = 'CALCIUM' then valuenum else null end) as CALCIUM
# , max(case when label = 'GLUCOSE' then valuenum else null end) as GLUCOSE
# , max(case when label = 'HEMATOCRIT' then valuenum else null end) as HEMATOCRIT
# , max(case when label = 'HEMOGLOBIN' then valuenum else null end) as HEMOGLOBIN
# , max(case when label = 'INTUBATED' then valuenum else null end) as INTUBATED
# , max(case when label = 'LACTATE' then valuenum else null end) as LACTATE
# , max(case when label = 'METHEMOGLOBIN' then valuenum else null end) as METHEMOGLOBIN
# , max(case when label = 'O2FLOW' then valuenum else null end) as O2FLOW
# , max(case when label = 'FIO2' then valuenum else null end) as FIO2
# , max(case when label = 'SO2' then valuenum else null end) as SO2 -- OXYGENSATURATION
# , max(case when label = 'PCO2' then valuenum else null end) as PCO2
# , max(case when label = 'PEEP' then valuenum else null end) as PEEP
# , max(case when label = 'PH' then valuenum else null end) as PH
# , max(case when label = 'PO2' then valuenum else null end) as PO2
# , max(case when label = 'POTASSIUM' then valuenum else null end) as POTASSIUM
# , max(case when label = 'REQUIREDO2' then valuenum else null end) as REQUIREDO2
# , max(case when label = 'SODIUM' then valuenum else null end) as SODIUM
# , max(case when label = 'TEMPERATURE' then valuenum else null end) as TEMPERATURE
# , max(case when label = 'TIDALVOLUME' then valuenum else null end) as TIDALVOLUME
# , max(case when label = 'VENTILATIONRATE' then valuenum else null end) as VENTILATIONRATE
# , max(case when label = 'VENTILATOR' then valuenum else null end) as VENTILATOR
# from
# ( -- begin query that extracts the data
#   select ie.subject_id, ie.hadm_id, ie.icustay_id
#   -- here we assign labels to ITEMIDs
#   -- this also fuses together multiple ITEMIDs containing the same data
#       , case
#         when itemid = 50800 then 'SPECIMEN'
#         when itemid = 50801 then 'AADO2'
#         when itemid = 50802 then 'BASEEXCESS'
#         when itemid = 50803 then 'BICARBONATE'
#         when itemid = 50804 then 'TOTALCO2'
#         when itemid = 50805 then 'CARBOXYHEMOGLOBIN'
#         when itemid = 50806 then 'CHLORIDE'
#         when itemid = 50808 then 'CALCIUM'
#         when itemid = 50809 then 'GLUCOSE'
#         when itemid = 50810 then 'HEMATOCRIT'
#         when itemid = 50811 then 'HEMOGLOBIN'
#         when itemid = 50812 then 'INTUBATED'
#         when itemid = 50813 then 'LACTATE'
#         when itemid = 50814 then 'METHEMOGLOBIN'
#         when itemid = 50815 then 'O2FLOW'
#         when itemid = 50816 then 'FIO2'
#         when itemid = 50817 then 'SO2' -- OXYGENSATURATION
#         when itemid = 50818 then 'PCO2'
#         when itemid = 50819 then 'PEEP'
#         when itemid = 50820 then 'PH'
#         when itemid = 50821 then 'PO2'
#         when itemid = 50822 then 'POTASSIUM'
#         when itemid = 50823 then 'REQUIREDO2'
#         when itemid = 50824 then 'SODIUM'
#         when itemid = 50825 then 'TEMPERATURE'
#         when itemid = 50826 then 'TIDALVOLUME'
#         when itemid = 50827 then 'VENTILATIONRATE'
#         when itemid = 50828 then 'VENTILATOR'
#         else null
#         end as label
#         , charttime
#         , value
#         -- add in some sanity checks on the values
#         , case
#           when valuenum <= 0 then null
#           when itemid = 50810 and valuenum > 100 then null -- hematocrit
#           when itemid = 50816 and valuenum > 100 then null -- FiO2
#           when itemid = 50817 and valuenum > 100 then null -- O2 sat
#           when itemid = 50815 and valuenum >  70 then null -- O2 flow
#           when itemid = 50821 and valuenum > 800 then null -- PO2
#            -- conservative upper limit
#         else valuenum
#         end as valuenum

#     from mimiciii.icustays ie
#     left join mimiciii.labevents le
#       on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
#       and le.charttime between (ie.intime - interval '6' hour) and (ie.intime + interval '2' day)
#       and le.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
#       and le.ITEMID in
#       -- blood gases
#       (
#         50800, 50801, 50802, 50803, 50804, 50805, 50806, 50807, 50808, 50809
#         , 50810, 50811, 50812, 50813, 50814, 50815, 50816, 50817, 50818, 50819
#         , 50820, 50821, 50822, 50823, 50824, 50825, 50826, 50827, 50828
#         , 51545
#       )
# ) pvt
# group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME
# order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME; """.format(tuple(patients))
    
# bg_all48_df=pd.read_sql_query(bg_all48_sql,conn)    

# #changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# # into the join at the end. removed the create table at beginning and comitt at the end. 
# #also select * from the initial generated table
# #also also changed the date range to 48 hours after, rather than 24. (2 day)


# ###guessing this isn't correct yet. need to adjust by removing the group_by's. addin charttime, and remove the max/min categories.

time: 50.9 s


In [21]:
#vital

vital_all48_sql = query_schema + """

SELECT *

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id, ce.charttime --added ce.charttime
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 'HeartRate' -- HeartRate
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 'SysBP' -- SysBP
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 'DiasBP' -- DiasBP
    when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 'MeanArtPress' -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 'RespRate' -- RespRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 'TempF' -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 'TempC' -- TempC
    when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 'SpO2' -- SpO2
    when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then 'Glucose' -- Glucose

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from mimiciii.icustays ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime and ie.intime + interval '2' day
  and ie.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --    Arterial BP [Systolic]
  442, --   Manual BP [Systolic]
  455, --   NBP [Systolic]
  6701, --  Arterial BP #2 [Systolic]
  220179, --    Non Invasive Blood Pressure systolic
  220050, --    Arterial Blood Pressure systolic

  8368, --  Arterial BP [Diastolic]
  8440, --  Manual BP [Diastolic]
  8441, --  NBP [Diastolic]
  8555, --  Arterial BP #2 [Diastolic]
  220180, --    Non Invasive Blood Pressure diastolic
  220051, --    Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --  Arterial BP Mean #2
  443, --   Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--    Respiratory Rate
  615,--    Resp Rate (Total)
  220210,-- Respiratory Rate
  224690, --    Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab and fingerstick
  807,--    Fingerstick Glucose
  811,--    Glucose (70-105)
  1529,--   Glucose
  3745,--   BloodGlucose
  3744,--   Blood Glucose
  225664,-- Glucose finger stick
  220621,-- Glucose (serum)
  226537,-- Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,  -- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --    "Temperature F"

  )
) pvt;
--group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
--order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;""".format(tuple(patients))
    
vital_all48_df=pd.read_sql_query(vital_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



time: 3min 5s


In [176]:
len(vital_all48_df)

6741975

time: 32.7 ms


In [197]:
vital_all48_df.head()

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum
0,21,111970,216859,2135-02-01 15:30:00,MeanBP,61.0
1,25,129635,203487,2160-11-02 15:00:00,MeanBP,82.0
2,25,129635,203487,2160-11-02 16:00:00,MeanBP,92.0
3,25,129635,203487,2160-11-02 17:00:00,MeanBP,71.0
4,25,129635,203487,2160-11-03 08:00:00,MeanBP,74.0


time: 168 ms


In [ ]:
#height
##not needed according to nelson


height_all48_sql = query_schema + """
""".format(tuple(patients))
    
height_all48_df=pd.read_sql_query(height_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#weight
#this doesn't need to be changed since we only need weight_admit according to nelson

weight_all48_sql = query_schema + """

""".format(tuple(patients))
    
weight_all48_df=pd.read_sql_query(weight_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#urine
##not needed according to nelson

urine_all48_sql = query_schema + """

""".format(tuple(patients))
    
urine_all48_df=pd.read_sql_query(urine_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#rrt
##not needed according to nelson

In [173]:
# #vital, mid-adjustment. saved for reference. 

# vital_all48_sql = query_schema + """
# SELECT *

# FROM  (
#   select ie.subject_id, ie.hadm_id, ie.icustay_id
#   , case
#     when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
#     when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 2 -- SysBP
#     when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 3 -- DiasBP
#     when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 4 -- MeanBP
#     when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 5 -- RespRate
#     when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 6 -- TempF, converted to degC in valuenum call
#     when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 6 -- TempC
#     when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 7 -- SpO2
#     when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then 8 -- Glucose

#     else null end as VitalID
#       -- convert F to C
#   , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

#   from mimiciii.icustays ie
#   left join mimiciii.chartevents ce
#   on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
#   and ce.charttime between ie.intime and ie.intime + interval '2' day
#   and ie.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
#   -- exclude rows marked as error
#   and ce.error IS DISTINCT FROM 1
#   where ce.itemid in
#   (
#   -- HEART RATE
#   211, --"Heart Rate"
#   220045, --"Heart Rate"

#   -- Systolic/diastolic

#   51, --    Arterial BP [Systolic]
#   442, --   Manual BP [Systolic]
#   455, --   NBP [Systolic]
#   6701, --  Arterial BP #2 [Systolic]
#   220179, --    Non Invasive Blood Pressure systolic
#   220050, --    Arterial Blood Pressure systolic

#   8368, --  Arterial BP [Diastolic]
#   8440, --  Manual BP [Diastolic]
#   8441, --  NBP [Diastolic]
#   8555, --  Arterial BP #2 [Diastolic]
#   220180, --    Non Invasive Blood Pressure diastolic
#   220051, --    Arterial Blood Pressure diastolic


#   -- MEAN ARTERIAL PRESSURE
#   456, --"NBP Mean"
#   52, --"Arterial BP Mean"
#   6702, --  Arterial BP Mean #2
#   443, --   Manual BP Mean(calc)
#   220052, --"Arterial Blood Pressure mean"
#   220181, --"Non Invasive Blood Pressure mean"
#   225312, --"ART BP mean"

#   -- RESPIRATORY RATE
#   618,--    Respiratory Rate
#   615,--    Resp Rate (Total)
#   220210,-- Respiratory Rate
#   224690, --    Respiratory Rate (Total)


#   -- SPO2, peripheral
#   646, 220277,

#   -- GLUCOSE, both lab and fingerstick
#   807,--    Fingerstick Glucose
#   811,--    Glucose (70-105)
#   1529,--   Glucose
#   3745,--   BloodGlucose
#   3744,--   Blood Glucose
#   225664,-- Glucose finger stick
#   220621,-- Glucose (serum)
#   226537,-- Glucose (whole blood)

#   -- TEMPERATURE
#   223762, -- "Temperature Celsius"
#   676,  -- "Temperature C"
#   223761, -- "Temperature Fahrenheit"
#   678 --    "Temperature F"

#   )
# ) pvt,
# --group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
# --order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
# """.format(tuple(patients))
    
# vital_all48_df=pd.read_sql_query(vital_all48_sql,conn)    

# #changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# # into the join at the end. removed the create table at beginning and comitt at the end. 
# #also select * from the initial generated table
# #also also changed the date range to 48 hours after, rather than 24. (2 day)





# ######this isn't quite there, need to adjust to grab all values and get time identifier for them. 

time: 5min 31s


In [196]:
pd.DataFrame(vital_all48_df).to_csv('clinical_csv/vital_all48_df.csv')
#pd.DataFrame(labs_all48_df).to_csv('clinical_csv/labs_all48_df.csv')
#pd.DataFrame(bg_all48_df).to_csv('clinical_csv/bg_all48_df.csv')


time: 1min 3s


# 5/11/18 todo:
1) figure out how to extract the additional features nelson want's

2) determine how to process each datapoint for single value for each feature in model

3) assemble df for scaling, imputing, model building, etc...


In [ ]:
#testing
#want to figure out easy way to run all my sql code without just pasting it into jupyter like i have been

In [9]:
#labs
labs_all48_sql = query_schema + """

SELECT *

FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, le.charttime
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
	  --WHEN ie.subject_id not in (24915,24942,24968,24776,24714,24824) THEN null
    ELSE le.valuenum
    END AS valuenum

  FROM mimiciii.icustays ie
	--where ie.subject_id in (24915,24942,24968,24776,24714,24824)
  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (ie.intime - interval '6' hour) AND (ie.intime + interval '2' day)
	AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt;

    """.format(tuple(patients))
    
labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    


#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)
#will need to do this for all of my sql queres.


###guessing this isn't correct yet. need to adjust by removing the group_by's. addin charttime, and remove the max/min categories.

time: 57.6 s


In [65]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/notebooks/sqltest.sql', 'r') 
labdata = f.read()
labs_all48_sql = query_schema + labdata.format(tuple(patients))
labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    

time: 33.1 s


In [75]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/gcs_all48.sql', 'r') 
gcsdata = f.read()
gcs_all48_sql = query_schema + gcsdata.format(tuple(icustay_id))
#gcs_all48_sql = query_schema + gcsdata.format(tuple(patients))
gcs_all48_df=pd.read_sql_query(gcs_all48_sql,conn)  

#changed daterange to 2, added in {} in the 2d select statement, removed first 2 lines of materialized view.



#added a patient filter at the end of the query, this isn't as efficient as if it were earlier but it was breaking when i tried that.

#removed the query_schema

DatabaseError: Execution failed on sql 'SET search_path to mimiciii;-- ITEMIDs used:

-- CAREVUE
--    723 as GCSVerbal
--    454 as GCSMotor
--    184 as GCSEyes

-- METAVISION
--    223900 GCS - Verbal Response
--    223901 GCS - Motor Response
--    220739 GCS - Eye Opening

-- The code combines the ITEMIDs into the carevue itemids, then pivots those
-- So 223900 is changed to 723, then the ITEMID 723 is pivoted to form GCSVerbal

-- Note:
--  The GCS for sedated patients is defaulted to 15 in this code.
--  This is in line with how the data is meant to be collected.
--  e.g., from the SAPS II publication:
--    For sedated patients, the Glasgow Coma Score before sedation was used.
--    This was ascertained either from interviewing the physician who ordered the sedation,
--    or by reviewing the patient's medical record.

--DROP MATERIALIZED VIEW IF EXISTS gcsfirstday CASCADE;
--create materialized view gcsfirstday as
--with base as
--DROP MATERIALIZED VIEW IF EXISTS gcsfirstday C

time: 52.3 ms


# glasco coma scale

In [174]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/gcs_all48.sql', 'r') 
gcsdata = f.read()
gcs_all48_sql = gcsdata.format(tuple(icustay_id))
cur.execute('rollback')
cur.execute(gcs_all48_sql)
gcsTable = sql_exe_show('select * from gcs_all48;')
gcs_all48_df= sql_to_df_icu(gcsTable)

time: 1min 39s


In [175]:
gcs_all48_sql

"-- ITEMIDs used:\n\n-- CAREVUE\n--    723 as GCSVerbal\n--    454 as GCSMotor\n--    184 as GCSEyes\n\n-- METAVISION\n--    223900 GCS - Verbal Response\n--    223901 GCS - Motor Response\n--    220739 GCS - Eye Opening\n\n-- The code combines the ITEMIDs into the carevue itemids, then pivots those\n-- So 223900 is changed to 723, then the ITEMID 723 is pivoted to form GCSVerbal\n\n-- Note:\n--  The GCS for sedated patients is defaulted to 15 in this code.\n--  This is in line with how the data is meant to be collected.\n--  e.g., from the SAPS II publication:\n--    For sedated patients, the Glasgow Coma Score before sedation was used.\n--    This was ascertained either from interviewing the physician who ordered the sedation,\n--    or by reviewing the patient's medical record.\n\n--DROP MATERIALIZED VIEW IF EXISTS gcsfirstday CASCADE;\n--create materialized view gcsfirstday as\n--with base as\n--DROP MATERIALIZED VIEW IF EXISTS gcsfirstday CASCADE;\n--create materialized view gcsfi

time: 3.92 ms


# bloodgas

In [106]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/bg_all48.sql', 'r') 
bgdata = f.read()
bg_all48_sql = bgdata.format(tuple(patients))
cur.execute('rollback')
cur.execute(bg_all48_sql)
bgTable = sql_exe_show('select * from bg_all48;')
bg_all48_df= sql_to_df_icu(bgTable)

time: 5min 4s


In [159]:
bg_all48_df.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum
2,4,185777,294638,2191-03-15 21:23:00,BASEEXCESS,NaN
3,4,185777,294638,2191-03-15 21:23:00,FIO2,40.0
4,4,185777,294638,2191-03-15 21:23:00,HEMATOCRIT,32.0
5,4,185777,294638,2191-03-15 21:23:00,HEMOGLOBIN,10.5
6,4,185777,294638,2191-03-15 21:23:00,INTUBATED,NaN


time: 10.3 ms


# labs

In [138]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/labs_all48.sql', 'r') 
labsdata = f.read()
labs_all48_sql = labsdata.format(tuple(patients))
#labs_all48_sql = labsdata.format(tuple(icustay_id))
#labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    
cur.execute('rollback')
cur.execute(labs_all48_sql)
labsTable = sql_exe_show('select * from labs_all48;')
labs_all48_df= sql_to_df_icu(labsTable)   

#toook 5 min to run with icustay_id filtered out in sql script.
#took 3min 40sec to run with no filters in sql and all filtering in python after the fact. 821973 rows
#took 51.9 sec to run with patient filtered out in sql script.  this is the winner

time: 51.9 s


# vitals

In [145]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/vitals_all48.sql', 'r') 
vitalsdata = f.read()
vitals_all48_sql = vitalsdata.format(tuple(patients))
#labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    
cur.execute('rollback')
cur.execute(vitals_all48_sql)
vitalsTable = sql_exe_show('select * from vitals_all48;')
vitals_all48_df= sql_to_df_icu(vitalsTable)

time: 7min 19s


# vent durations

In [152]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/ventsettings_all48.sql', 'r') 
ventsettingsdata = f.read()
ventsettings_all48_sql = ventsettingsdata.format(tuple(patients),tuple(icustay_id))  
cur.execute('rollback')
cur.execute(ventdurations_all48_sql)
ventdurationsTable= sql_exe_show('select * from ventdurations_all48;')
ventdurations_all48_df= sql_to_df_icu(ventdurationsTable)


time: 9min 2s


In [158]:
ventdurations_all48_df.head()

,icustay_id,ventnum,starttime,endtime,duration_hours
0,200019,1,2178-07-08 10:00:00,2178-07-09 13:00:00,27.000000
1,200059,1,2198-02-10 06:00:00,2198-02-17 11:00:00,173.000000
2,200059,2,2198-02-17 15:00:00,2198-02-17 16:00:00,1.000000
3,200059,3,2198-02-17 20:30:00,2198-02-18 07:00:00,10.500000
4,200059,4,2198-02-18 22:35:00,2198-02-19 07:00:00,8.416667


time: 7.4 ms


In [155]:
cur.execute(ventsettings_all48_sql)
ventsettingsTable = sql_exe_show('select * from ventsettings_all48;')
ventsettings_all48_df= sql_to_df_icu(ventsettingsTable)



time: 9min 42s


In [157]:
ventsettings_all48_df.head()



,icustay_id,charttime,mechvent,oxygentherapy,extubated,selfextubated
0,258300.0,2118-07-07 12:00:00,1,0,0,0
2,249035.0,2114-02-01 12:00:00,1,0,0,0
3,282983.0,2185-01-14 14:30:00,1,0,0,0
4,249403.0,2158-10-25 22:25:00,1,0,0,0
7,268546.0,2159-06-06 00:00:00,0,1,0,0


time: 8.39 ms


# vent parameters

In [162]:
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/ventparam_all48.sql', 'r') 
ventparamdata = f.read()
ventparam_all48_sql = ventparamdata.format(tuple(icustay_id))  
cur.execute('rollback')
cur.execute(ventparam_all48_sql)
ventparamTable= sql_exe_show('select * from ventparam_all48;')
ventparam_all48_df= sql_to_df_icu(ventparamTable)

time: 7min 28s


committing all data extracted week of 6/21/18 to csv (note file naming convention uses day/month/year). 

In [171]:
pd.DataFrame(ventparam_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_ventparam_all48_df.csv')
pd.DataFrame(ventsettings_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_ventsettings_all48_df.csv')
pd.DataFrame(vitals_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_vitals_all48_df.csv')
pd.DataFrame(labs_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_labs_all48_df.csv')
pd.DataFrame(bg_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_bg_all48_df.csv')
pd.DataFrame(gcs_all48_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_gcs_all48_df.csv')

time: 1min 21s


In [170]:
ventparam_all48_df.loc[ventparam_all48_df['label']=='PEEP',:]['itemid'].value_counts()



220339    217866
506       163566
224700     19384
60          2121
686         1701
437          777
505            5
Name: itemid, dtype: int64

time: 353 ms


In [ ]:
#todo: look at distribution of each item id. can also look at actual label corresponding to each itemid. 

#also need to annotate the itemid from the d_item table. 

In [166]:
ventparam_all48_df.loc[ventparam_all48_df[,'label']=='PEEP',:]


,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,label
0,34768101,111,192123,254245,60,2142-04-25 00:30:00,2142-04-25 00:36:00,19653.0,4,4.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
1,34770020,111,192123,254245,60,2142-04-26 20:00:00,2142-04-26 20:58:00,17698.0,0,0.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
2,34768413,111,192123,254245,60,2142-04-25 06:00:00,2142-04-25 05:51:00,19653.0,1,1.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
3,34770300,111,192123,254245,60,2142-04-27 07:00:00,2142-04-27 06:33:00,14381.0,0,0.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
4,34769502,111,192123,254245,60,2142-04-26 07:18:00,2142-04-26 07:21:00,16985.0,.80000001192092896,0.8,cmH2O,NaN,NaN,None,NotStopd,PEEP
5,34767800,111,192123,254245,60,2142-04-24 20:00:00,2142-04-24 23:25:00,19653.0,0,0.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
6,34769934,111,192123,254245,60,2142-04-26 18:30:00,2142-04-26 18:29:00,17698.0,4,4.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
7,34768329,111,192123,254245,60,2142-04-25 04:00:00,2142-04-25 05:51:00,19653.0,1,1.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
8,34770242,111,192123,254245,60,2142-04-27 04:00:00,2142-04-27 04:47:00,17698.0,2,2.0,cmH2O,NaN,NaN,None,NotStopd,PEEP
9,34769090,111,192123,254245,60,2142-04-25 23:00:00,2142-04-25 22:54:00,19653.0,0,0.0,cmH2O,NaN,NaN,None,NotStopd,PEEP


time: 50.6 ms


In [ ]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df